In [1]:
pip install inference_sdk

  Obtaining dependency information for inference_sdk from https://files.pythonhosted.org/packages/6b/e4/46ce33595aa4ca65c99205e4739b033c08d376af29dc51bdf5da4e96f5e8/inference_sdk-0.9.17-py3-none-any.whl.metadata
  Obtaining dependency information for dataclasses-json>=0.6.0 from https://files.pythonhosted.org/packages/91/ca/7219b838086086972e662c19e908694bdc6744537fb41b70392501b8b5e4/dataclasses_json-0.6.4-py3-none-any.whl.metadata
  Obtaining dependency information for supervision<1.0.0 from https://files.pythonhosted.org/packages/9c/8a/8e86cb06d6e10d532c54722977d2b06797026336729ce37a2d35504f23d6/supervision-0.19.0-py3-none-any.whl.metadata
  Obtaining dependency information for aiohttp>=3.9.0 from https://files.pythonhosted.org/packages/a3/59/cd5e456835df696e6307c8b045acfada1557ac822d527c27bb33c1308091/aiohttp-3.9.3-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for backoff>=2.2.0 from https://files.pythonhosted.org/packages/df/73/b6e24bd22e6720ca8ee9a85a0c4a29

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.
aiobotocore 2.5.0 requires botocore<1.29.77,>=1.29.76, but you have botocore 1.31.46 which is incompatible.
s3fs 2023.4.0 requires fsspec==2023.4.0, but you have fsspec 2024.2.0 which is incompatible.


In [15]:
import pandas as pd
import cv2
import numpy as np
from sklearn.cluster import KMeans
import os

# import the inference-sdk
from inference_sdk import InferenceHTTPClient

### Importing the input images

In [87]:
folder_path = "integrationTest/input"
filenames = []
for filename in os.listdir(folder_path):
# Check if it's a file (not a directory)
    if os.path.isfile(os.path.join(folder_path, filename)):
        filenames.append(folder_path + '/' + filename)
filenames.remove('integrationTest/input/desktop.ini')

filenames

['integrationTest/input/121986373_3.jpg',
 'integrationTest/input/165173803_2.jpg',
 'integrationTest/input/165400207_10.jpg',
 'integrationTest/input/180490298_2.jpg',
 'integrationTest/input/180670199_2.jpg',
 'integrationTest/input/181496613_2.jpg',
 'integrationTest/input/198682647_2.jpg',
 'integrationTest/input/198815950_2.jpg',
 'integrationTest/input/22236070_3.jpg',
 'integrationTest/input/27801560_4.jpg',
 'integrationTest/input/28516419_4.jpg',
 'integrationTest/input/95964270_1.jpg',
 'integrationTest/input/96411715_1.jpg',
 'integrationTest/input/96531369_1.jpg']

In [64]:
column_names = ['FileName', 'ID', 'Color', 'Casual', 'Formal', 'Party']

df = pd.DataFrame(columns=column_names)

df

,FileName,ID,Color,Casual,Formal,Party


### Detecting the Type and Occasion

In [35]:
# initialize the client
CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key="n1AauQE75rXb406ZDdyM"
)

# infer on a local image
result = CLIENT.infer("integrationTest/input/121986373_3.jpg", model_id="occasion-detection/1")['predicted_classes']
result

['casual', 'shoes']

In [54]:
'casual' in result
# result

True

In [72]:
type_list = ['upper', 'lower', 'over', 'shoes']

type_dictionary = {
    'upper' : 0,
    'lower' : 1,
    'over' : 2,
    'shoes': 3
}

In [74]:
type_dictionary['upper']

0

In [90]:
def detectTypeOccasion(img):
    # initialize the client
    CLIENT = InferenceHTTPClient(
        api_url="https://detect.roboflow.com",
        api_key="n1AauQE75rXb406ZDdyM"
    )
    
    # infer on a local image
    result = CLIENT.infer(img, model_id="occasion-detection/1")['predicted_classes']
    
    for item in type_list:
        if item in result:
            ID = type_dictionary[item]
    
    casual = 1 if 'casual' in result else 0
    party = 1 if 'party' in result else 0
    formal = 1 if 'formal' in result else 0
#     ID = 0
    
    print(result)
    return casual, formal, party, ID

In [5]:
result

{'time': 0.18686104100004286,
 'image': {'width': 305, 'height': 400},
 'predictions': {'casual': {'confidence': 0.8150017261505127},
  'formal': {'confidence': 0.08387866616249084},
  'lower': {'confidence': 0.1014958918094635},
  'over': {'confidence': 0.10006295889616013},
  'party': {'confidence': 0.3742920160293579},
  'shoes': {'confidence': 0.7602288722991943},
  'sport': {'confidence': 0.1554105132818222},
  'upper': {'confidence': 0.12531113624572754}},
 'predicted_classes': ['casual', 'shoes']}

### Detecting the Color

In [169]:
def detectColor(img):
    # Load the image
    image = cv2.imread(img)

    # Convert the image to HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Flatten the image into a 2D array of pixels
    pixels = hsv_image.reshape((-1, 3))

    # Define the number of clusters (colors) you want to detect
    num_clusters = 2 # Adjust this based on the number of colors you expect

    # Apply K-means clustering
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(pixels)

    # Get the representative colors (centroids) of each cluster
    colors = kmeans.cluster_centers_

    # Convert the colors from float to integer
    colors = colors.astype(np.uint8)

    # Optionally, convert the colors to BGR and display them
    # colors_bgr = colors
    colors_rgb = cv2.cvtColor(colors.reshape(1, -1, 3), cv2.COLOR_HSV2RGB)
    #     colors_bgr = cv2.cvtColor(colors.reshape(1, -1, 3), cv2.COLOR_BGR2LAB)
    
    return colors_rgb[0][0]

### Filling the Data Frame

In [132]:
folder_path = "integrationTest/input"
filenames = []
for filename in os.listdir(folder_path):
# Check if it's a file (not a directory)
    if os.path.isfile(os.path.join(folder_path, filename)):
        filenames.append(folder_path + '/' + filename)
filenames.remove('integrationTest/input/desktop.ini')

filenames

['integrationTest/input/121986373_3.jpg',
 'integrationTest/input/165173803_2.jpg',
 'integrationTest/input/165400207_10.jpg',
 'integrationTest/input/180490298_2.jpg',
 'integrationTest/input/180670199_2.jpg',
 'integrationTest/input/181496613_2.jpg',
 'integrationTest/input/198682647_2.jpg',
 'integrationTest/input/198815950_2.jpg',
 'integrationTest/input/22236070_3.jpg',
 'integrationTest/input/27801560_4.jpg',
 'integrationTest/input/28516419_4.jpg',
 'integrationTest/input/6248157_1.jpg',
 'integrationTest/input/95964270_1.jpg',
 'integrationTest/input/96411715_1.jpg',
 'integrationTest/input/96531369_1.jpg']

In [200]:
column_names = ['FileName', 'ID', 'HSV', 'Casual', 'Formal', 'Party']

df = pd.DataFrame(columns=column_names)

for files in filenames:
    # detectColor('integrationTest/input/121986373_3.jpg')
    color = detectColor(files)
    casual, formal, party, ID = detectTypeOccasion(files)
    
    row = {
        'FileName': files,
        'ID' : ID,
        'HSV' : color,
        'Casual' : casual,
        'Formal' : formal,
        'Party' : party
    }
    
#     df = df.append(new_row, ignore_index=True)
    df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
    
df

['casual', 'shoes']
['casual', 'lower']
['lower', 'party']
['casual', 'over']
['over']
['formal', 'over', 'party']
['lower', 'party']
['lower', 'party']
['casual', 'shoes']
['casual', 'party', 'shoes']
['party', 'shoes']
['formal', 'upper']
['casual', 'party', 'upper']
['party', 'upper']
['formal', 'over', 'party']


,FileName,ID,HSV,Casual,Formal,Party
0,integrationTest/input/121986373_3.jpg,3,"[245, 243, 240]",1,0,0
1,integrationTest/input/165173803_2.jpg,1,"[251, 251, 251]",1,0,0
2,integrationTest/input/165400207_10.jpg,1,"[21, 17, 23]",0,0,1
3,integrationTest/input/180490298_2.jpg,2,"[53, 51, 56]",1,0,0
4,integrationTest/input/180670199_2.jpg,2,"[253, 252, 246]",0,0,0
5,integrationTest/input/181496613_2.jpg,2,"[252, 252, 252]",0,1,1
6,integrationTest/input/198682647_2.jpg,1,"[119, 41, 35]",0,0,1
7,integrationTest/input/198815950_2.jpg,1,"[245, 79, 29]",0,0,1
8,integrationTest/input/22236070_3.jpg,3,"[246, 245, 245]",1,0,0
9,integrationTest/input/27801560_4.jpg,3,"[241, 238, 235]",1,0,1


In [201]:

# #converting RGB to HSV
def rgb_to_hsv(r, g, b,precision=2):
    r, g, b = r / 255.0, g / 255.0, b / 255.0
    max_rgb = max(r, g, b)
    min_rgb = min(r, g, b)
    delta = max_rgb - min_rgb

    if delta == 0:
        h = 0
    elif max_rgb == r:
        h = (60 * ((g - b) / delta) + 360) % 360
    elif max_rgb == g:
        h = (60 * ((b - r) / delta) + 120) % 360
    else:
        h = (60 * ((r - g) / delta) + 240) % 360

    if max_rgb == 0:
        s = 0
    else:
        s = delta / max_rgb

    v = max_rgb

    return round(h), round(s, precision), round(v, precision)

df['HSV'] = df['HSV'].apply(lambda x: rgb_to_hsv(*x))
df



,FileName,ID,HSV,Casual,Formal,Party
0,integrationTest/input/121986373_3.jpg,3,"(36, 0.02, 0.96)",1,0,0
1,integrationTest/input/165173803_2.jpg,1,"(0, 0.0, 0.98)",1,0,0
2,integrationTest/input/165400207_10.jpg,1,"(280, 0.26, 0.09)",0,0,1
3,integrationTest/input/180490298_2.jpg,2,"(264, 0.09, 0.22)",1,0,0
4,integrationTest/input/180670199_2.jpg,2,"(51, 0.03, 0.99)",0,0,0
5,integrationTest/input/181496613_2.jpg,2,"(0, 0.0, 0.99)",0,1,1
6,integrationTest/input/198682647_2.jpg,1,"(4, 0.71, 0.47)",0,0,1
7,integrationTest/input/198815950_2.jpg,1,"(14, 0.88, 0.96)",0,0,1
8,integrationTest/input/22236070_3.jpg,3,"(0, 0.0, 0.96)",1,0,0
9,integrationTest/input/27801560_4.jpg,3,"(30, 0.02, 0.95)",1,0,1


In [202]:
def is_analogous(hsv1, hsv2):
    # Calculate the absolute difference in hue
    hue_diff = abs(hsv1[0] - hsv2[0])
    # Adjust for the circular nature of the color wheel
    if hue_diff > 180:
        hue_diff = 360 - hue_diff
    # Check if the hues are within 30 degrees of each other
    return hue_diff <= 30

def is_harmonic(hsv1, hsv2):
    # Example harmonic function implementation']
    hue_diff = abs(hsv1[0] - hsv2[0])
    return hue_diff < 30 or hue_diff > 330

def is_complementary(hsv1, hsv2):
    # Example complementary function implementation
    hue_diff = abs(hsv1[0] - hsv2[0])
    return abs(hue_diff - 180) < 10

In [220]:
def FFind_unique_outfit(df, occasion=None, include_over=False):
    df_occ = df[df[occasion] == 1] if occasion else df
    if df_occ[df_occ['ID'] == 0].empty:
        return "No upper items available for selection."

    upper_items = df_occ[df_occ['ID'] == 0]
    lower_items = df_occ[df_occ['ID'] == 1]
    shoes = df_occ[df_occ['ID'] == 3]
    overs = df_occ[df_occ['ID'] == 2] if include_over else pd.DataFrame()

    selected_uppers = upper_items if len(upper_items) <= 3 else upper_items.sample(n=3, replace=False)
#     print(selected_uppers)
    color_relations = [is_harmonic, is_complementary, is_analogous]
    relation_names = ['Harmonic', 'Complementary', 'Analogous']
    outfits = []

    for _, upper in selected_uppers.iterrows():
        outfit_attempts = []
        attempts = 0
        print(upper)
        
        while len(outfit_attempts) < 3 and attempts < 10:
           # Check for uniqueness=

            relation = color_relations[attempts % len(color_relations)]
            relation_name = relation_names[attempts % len(color_relations)]
            upper_hsv = upper['HSV']


            shoe_candidates = shoes[shoes['HSV'].apply(lambda x: relation(x, upper_hsv))]
            lower_candidates = lower_items[lower_items['HSV'].apply(lambda x: relation(x, upper_hsv))] 
            over_candidates = overs[overs['HSV'].apply(lambda x: relation(x, upper_hsv))] if include_over else pd.DataFrame()

            if not shoe_candidates.empty:
                shoe = shoe_candidates.sample(n=1)
            else:
                attempts += 1
                continue

            if not lower_candidates.empty:
                lower = lower_candidates.sample(n=1)  
            else:
                attempts += 1
                continue

            if include_over and not over_candidates.empty:
                over = over_candidates.sample(n=1)
            else:
                over = pd.DataFrame()

            # Construct a tentative outfit
            tentative_outfit = [("Upper", upper['FileName'], upper['HSV'])]
            if not shoe.empty:
                tentative_outfit.append(("Shoes", shoe.iloc[0]['FileName'], shoe.iloc[0]['HSV']))
            if not lower.empty:
                tentative_outfit.append(("Lower", lower.iloc[0]['FileName'], lower.iloc[0]['HSV']))
            if not over.empty:
                tentative_outfit.append(("Over", over.iloc[0]['FileName'], over.iloc[0]['HSV']))

            outfit_tuple = tuple([item for item in tentative_outfit])



            # Check for uniqueness
            if outfit_tuple not in outfit_attempts:
                outfit_attempts.append(outfit_tuple)
                outfits.append(tentative_outfit)  # Add to final outfits list as the formatted structure

            attempts += 1

    # Format the output to include category, color, and relation type for each outfit item
    formatted_outfits = []
    for outfit in outfits:
        formatted_outfit = {}
        for item in outfit:
            formatted_outfit[item[0]] = {'FileName': item[1], 'HSV': item[2]}
        formatted_outfits.append(formatted_outfit)

    return formatted_outfits


In [222]:
FFind_unique_outfit(df, occasion='Party', include_over=False)

FileName    integrationTest/input/95964270_1.jpg
ID                                             0
HSV                               (7, 0.04, 0.8)
Casual                                         1
Formal                                         0
Party                                          1
Name: 12, dtype: object
FileName    integrationTest/input/96411715_1.jpg
ID                                             0
HSV                               (60, 0.0, 1.0)
Casual                                         0
Formal                                         0
Party                                          1
Name: 13, dtype: object


[{'Upper': {'FileName': 'integrationTest/input/95964270_1.jpg',
   'HSV': (7, 0.04, 0.8)},
  'Shoes': {'FileName': 'integrationTest/input/27801560_4.jpg',
   'HSV': (30, 0.02, 0.95)},
  'Lower': {'FileName': 'integrationTest/input/198682647_2.jpg',
   'HSV': (4, 0.71, 0.47)}},
 {'Upper': {'FileName': 'integrationTest/input/95964270_1.jpg',
   'HSV': (7, 0.04, 0.8)},
  'Shoes': {'FileName': 'integrationTest/input/27801560_4.jpg',
   'HSV': (30, 0.02, 0.95)},
  'Lower': {'FileName': 'integrationTest/input/198815950_2.jpg',
   'HSV': (14, 0.88, 0.96)}}]